<a href="https://colab.research.google.com/github/konstihengge/Tensorflow_tutorial/blob/main/Tensorflow_tut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.version)

t = tf.ones([5,5,5,5])

t = tf.reshape(t, [125, -1])


In [ ]:
#Regression


!pip install -q sklearn
%tensorflow_version 2.x  # this line is not required unless you are in a notebook

from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

# Load dataset.
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # training data
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
print(dfeval.shape)
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')
print(dftrain.loc[0], dftrain.loc[100]) # damit kann dann eine bestimmte Zeile im Datensatz ermittelt werden

CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']


#Feature Collums
#------------------------------------------------------------------------------------------------------------------------
feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()  # gets a list of all unique values from given feature column; es werden alle einzigartigen werte Herausgenommen
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary)) # fuegt alles in eine Zeile zusammen

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)


#Input Function
#--------------------------------------------------------------------------------------------------------------------------
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

train_input_fn = make_input_fn(dftrain, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)




#Creating the model
#-----------------------------------------------------------------------------------------------------------------------------
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
# We create a linear estimtor by passing the feature columns we created earlier



#Trining the model
#------------------------------------------------------------------------------------------------------------------------------
linear_est.train(train_input_fn)  # train
result = linear_est.evaluate(eval_input_fn)  # get model metrics/stats by testing on tetsing data

clear_output()  # clears consoke output
print(result['accuracy'])  # the result variable is simply a dict of stats about our model
print(result)





In [ ]:
result = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[4])
print(y_eval.loc[4])
print(result[4]["probabilities"][1]) #[1] bedeutet das es die Wahrscheinlichkeit zum überleben ist; [0] bedeuted wahrscheinlnichkeit das sterben

**Classification**

In [ ]:
#Classification



%tensorflow_version 2.x

from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import pandas as pd


#Dataset
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
# Lets define some constants to help us later on

train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

train_y = train.pop('Species')
test_y = test.pop('Species')




#Input function
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

#Feature Column
my_feature_columns = []
for key in train.keys():
   my_feature_columns.append(tf.feature_column.numeric_column(key=key))


#Building the modul
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)



#Traininig
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True), # lambda funktion erstellt, im Anschluss steht, was die Funktion macht), Lambda Funktin ersetzt in der Input-Function die äußerer Fuktion??
    steps=5000)
# We include a lambda to avoid creating an inner function previously



In [ ]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

In [ ]:
#Prediction
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size) # gibt keine Labels  

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {} # hier kann ein Datensatz eingefuegt werden

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid: 
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]   #bis hier werden alle features abgefragt und die Werte eingegeben, bis es keine Features mehr gibt

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    print(pred_dict)
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))
    



  # Here is some example input and expected classes you can try above
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

**Clustering**

Bedeutet das mehrer Punkte zu einem Cluster zusammengefasst werden und das immer wieder gemacht wird
-> so entstehen dann die Punkte

**Hidden Markov Models**

Das Hidden-Markov-Modell ist eine endliche Menge von Zuständen, von denen jeder mit einer (im Allgemeinen mehrdimensionalen) Wahrscheinlichkeitsverteilung [] verbunden ist. Übergänge zwischen den Zuständen werden durch eine Menge von Wahrscheinlichkeiten geregelt, die Übergangswahrscheinlichkeiten genannt werden." 

Ein Hidden-Markov-Modell arbeitet mit Wahrscheinlichkeiten, um zukünftige Ereignisse oder Zustände vorherzusagen. In diesem Abschnitt erfahren Sie, wie Sie ein Hidden-Markov-Modell erstellen, das das Wetter vorhersagen kann.

Zustände: In jedem Markov-Modell haben wir eine endliche Menge von Zuständen. Diese Zustände könnten etwa „warm“ und „kalt“ oder „hoch“ und „niedrig“ oder sogar „rot“, „grün“ und „blau“ sein. Diese Zustände sind im Modell "versteckt", das heißt, wir beobachten sie nicht direkt.

Beobachtungen: Jedem Zustand ist basierend auf einer Wahrscheinlichkeitsverteilung ein bestimmtes Ergebnis oder eine bestimmte Beobachtung zugeordnet. Ein Beispiel dafür ist das folgende: An einem heißen Tag hat Tim eine Chance von 80 %, glücklich und eine Chance von 20 %, traurig zu sein.

Übergänge: Jeder Zustand hat eine Wahrscheinlichkeit, die die Wahrscheinlichkeit des Übergangs in einen anderen Zustand definiert. Ein Beispiel ist das folgende: Auf einen kalten Tag kann mit einer Wahrscheinlichkeit von 30 % ein heißer Tag folgen und mit einer Wahrscheinlichkeit von 70 %, dass ein weiterer kalter Tag folgt.